In [12]:
%pip install lxml


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Temporadas a extraer
temporadas = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
base_url = "https://fbref.com/es/comps/35/{}/Estadisticas-{}-Chilean-Primera-Division"

# Lista para almacenar los datos
df_list = []

for temporada in temporadas:
    print(f"Obteniendo datos de la temporada {temporada}...")

    # Construcción de la URL específica para la temporada
    url = base_url.format(temporada, temporada)

    # Obtener la página
    response = requests.get(url)
    if response.status_code != 200:
        print(f"No se pudo acceder a la página para {temporada}")
        continue

    # Analizar el HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Buscar la tabla con id "stats_squads_standard_for"
    table_id = f"results{temporada}351_home_away"
    table = soup.find("table", {"id": table_id})
    
    if table is None:
        print(f"No se encontró la tabla para {temporada}, puede que la URL haya cambiado.")
        continue
    for tr in table.find_all('tr'):
        print(tr) 
    # Convertir la tabla a DataFrame de Pandas
    df = pd.read_html(str(table))[0]

    # Agregar la columna de la temporada
    df["Temporada"] = temporada

    # Almacenar en la lista
    df_list.append(df)

# Concatenar todas las temporadas en un solo DataFrame
if df_list:
    df_final = pd.concat(df_list, ignore_index=True)
    
    # Guardar en un CSV único
    df_final.to_csv("estadisticas_liga_chilena.csv", index=False, sep=";", encoding="utf-8-sig")
    print("Datos guardados en estadisticas_liga_chilena.csv")
else:
    print("No se obtuvo información de ninguna temporada.")


Obteniendo datos de la temporada 2017...
<tr class="over_header"> <th aria-label="" class="over_header center" colspan="2" data-stat=""></th> <th aria-label="" class="over_header center group_start" colspan="9" data-stat="header_home">Local</th> <th aria-label="" class="over_header center group_start" colspan="9" data-stat="header_away">Visitante</th> </tr>
<tr> <th aria-label="Rango" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rango&lt;/strong&gt;&lt;br&gt;Posición final del equipo en la competencia&lt;br&gt;Terminan dentro de la liga o competición.&lt;br&gt;Para las competiciones eliminatorias se puede mostrar la ronda final alcanzada.&lt;br&gt;Los colores y flechas representan los ascensos/descensos o clasificación para las copas continentales.&lt;br&gt;El trofeo indica que el equipo ganó la liga ya sea por eliminatorias o por liderar la tabla de clasificación.&lt;br&gt;La estrella indica que está en la primera posición de la tabla de la liga UTIL

In [6]:
import pandas as pd

# Cargar el CSV con múltiples encabezados
df = pd.read_csv("estadisticas_liga_chilena.csv",  sep=";", header=[0, 1])

# Unir encabezados múltiples (por ejemplo: 'Local', 'PJ' → 'Local_PJ')
df.columns = ['{}_{}'.format(a, b) if a != 'Unnamed: 0_level_0' and b != 'Equipo' else b
              for a, b in df.columns]

# Renombrar columna de posición
df.rename(columns={"RL": "Posicion_Final", "Equipo": "Equipo", "Temporada_": "Temporada"}, inplace=True)

# Convertir columnas necesarias a numéricas
columnas_numericas = [
    "Local_PJ", "Local_PG", "Local_PE", "Local_PP", "Local_GF", "Local_GC", "Local_DG", "Local_Pts",
    "Visitante_PJ", "Visitante_PG", "Visitante_PE", "Visitante_PP", "Visitante_GF", "Visitante_GC",
    "Visitante_DG", "Visitante_Pts"
]

for col in columnas_numericas:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Crear columnas combinadas (suma local + visitante)
df["PJ"] = df["Local_PJ"] + df["Visitante_PJ"]
df["PG"] = df["Local_PG"] + df["Visitante_PG"]
df["PE"] = df["Local_PE"] + df["Visitante_PE"]
df["PP"] = df["Local_PP"] + df["Visitante_PP"]
df["GF"] = df["Local_GF"] + df["Visitante_GF"]
df["GC"] = df["Local_GC"] + df["Visitante_GC"]
df["DG"] = df["Local_DG"] + df["Visitante_DG"]
df["Pts"] = df["Local_Pts"] + df["Visitante_Pts"]

# Crear la columna objetivo: 1 si fue campeón (posición 1), 0 en otro caso
df["Campeon"] = (df["Posicion_Final"] == 1).astype(int)
df.rename(columns={"Temporada_Unnamed: 20_level_1": "Temporada"}, inplace=True)

# Elegimos solo las columnas finales para el modelo
df_modelo = df[["Equipo", "Temporada", "PJ", "PG", "PE", "PP", "GF", "GC", "DG", "Pts", "Campeon"]]
df_modelo.to_csv("modelo_entrenamiento.csv", index=False)

# Mostrar ejemplo
print(df_modelo.head())




           Equipo  Temporada  PJ  PG  PE  PP  GF  GC  DG  Pts  Campeon
0       Colo-Colo       2017  15  10   3   2  33  13  20   33        1
1  Unión Española       2017  15   9   4   2  18  12   6   31        0
2      U de Chile       2017  15   9   3   3  21  18   3   30        0
3         Everton       2017  15   7   5   3  24  19   5   26        0
4  Audax Italiano       2017  15   7   4   4  24  19   5   25        0


In [11]:
# Importar librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Cargar el dataset
df = pd.read_csv('modelo_entrenamiento.csv')  # Asegúrate de poner la ruta correcta de tu archivo CSV

# Definir las características (X) y el objetivo (y)
X = df[['PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'DG', 'Pts']]  # Características
y = df['Campeon']  # Columna objetivo (campeon = 1, no campeon = 0)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo XGBoost con balance de clases
xgb = XGBClassifier(scale_pos_weight=15, random_state=42)  # scale_pos_weight ajusta el balance entre clases

# Entrenar el modelo
xgb.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de prueba
y_pred = xgb.predict(X_test)

# Evaluar el modelo con las métricas de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Evaluar la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, y_pred))


Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        26
           1       1.00      1.00      1.00         1

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27

Precisión del modelo:
1.0
